## OBJECTIVE : 

### To identify if tweets are about disaster or not (Kaggle Competition)

In [1]:
import warnings
warnings.simplefilter("ignore")

In [5]:
import os
import sys
import ipdb
import numpy as np
import pandas as pd

print("Numpy Version : ", np.__version__)
print("Pandas Version : ", pd.__version__)

np.random.seed(42)

%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

print("Matplotlib Version : ", mpl.__version__)

Numpy Version :  1.18.1
Pandas Version :  1.0.1
Matplotlib Version :  3.1.3


In [6]:
DATA_DIR = "../data/"
OBJECT_DIR = "../objects/"
LOG_DIR = "../logs/"

In [7]:
LUCKY_SEED = 42

In [8]:
ORIG_DATA_DIR = os.path.join(DATA_DIR, "orig_data/nlp-getting-started")
TRAIN_DIR = os.path.join(DATA_DIR, "training_data")
VALID_DIR = os.path.join(DATA_DIR, "validation_data")
TEST_DIR = os.path.join(DATA_DIR, "testing_data")

In [9]:
# Read data
data = pd.read_csv(os.path.join(ORIG_DATA_DIR, "train.csv"))

In [10]:
print("Shape of data : ", data.shape)

Shape of data :  (7613, 5)


In [14]:
print("Features : ", data.columns.values)

Features :  ['id' 'keyword' 'location' 'text' 'target']


In [15]:
data.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [16]:
print("Counts of different Targets : ")
data["target"].value_counts()

Counts of different Targets : 


0    4342
1    3271
Name: target, dtype: int64

In [20]:
print("Dataframe Information : ")
data.info()

Dataframe Information : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [21]:
print("Count of null values in text column : {}".format(data["text"].isnull().sum()))

Count of null values in text column : 0


In [22]:
print("Count of null values in target column : {}".format(data["target"].isnull().sum()))

Count of null values in target column : 0


In [23]:
# Read test data
test_data = pd.read_csv(os.path.join(ORIG_DATA_DIR, "test.csv"))

In [24]:
print("Shape of test data : {}".format(test_data.shape))

Shape of test data : (3263, 4)


In [25]:
print("Features in test data : {}".format(test_data.columns.values))

Features in test data : ['id' 'keyword' 'location' 'text']


In [26]:
test_data.head()

,id,keyword,location,text
0,0,NaN,NaN,Just happened a terrible car crash
1,2,NaN,NaN,"Heard about #earthquake is different cities, s..."
2,3,NaN,NaN,"there is a forest fire at spot pond, geese are..."
3,9,NaN,NaN,Apocalypse lighting. #Spokane #wildfires
4,11,NaN,NaN,Typhoon Soudelor kills 28 in China and Taiwan


In [27]:
print("Test Dataframe Information : ")
test_data.info()

Test Dataframe Information : 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3263 entries, 0 to 3262
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        3263 non-null   int64 
 1   keyword   3237 non-null   object
 2   location  2158 non-null   object
 3   text      3263 non-null   object
dtypes: int64(1), object(3)
memory usage: 102.1+ KB
